In [1]:
import pandas as pd
import numpy as  np
from IPython.display import Image
import dot2tex as d2t
import subprocess

import event_tree_class as tree
import tree2tex as t2t

# def export_to_tex(mod, filename):
#     mod_str = mod.to_string()
#     f = open("figures/%s.dot" %filename, "w+")
#     f.write(mod_str)
#     f.close()
#     cmd = "dot2tex -tmath --figonly figures/%s.dot > figures/%s.tex" % (filename, filename)
#     subprocess.Popen(cmd, shell=True)

# Frequency Model

In [2]:
df = pd.read_csv("../data/processed/freq_mod.csv", index_col=0)
df.head()

,age_group,sex,freq,acc_inv,hp
1,"(18,21]",M,2,0,0.0
2,"(21,80]",F,1,0,0.0
3,"(21,80]",M,1,0,0.0
4,"(21,80]",F,1,0,0.0
5,"(21,80]",F,3,0,0.0


In [4]:
order = ["age_group", "sex", "freq", "acc_inv"]
mod_freq = tree.ceg({'dataframe' : df[order]})
new_nodes = ["v_{" + str(n) + "}" for n in list(range(0, len(mod_freq.nodes)))]
mod_freq.rename_nodes(new_nodes)
mod_freq.AHC_transitions(alpha = 3)

mod_freq_tree = mod_freq.event_tree_figure(
    "figures/tree", 
    params={'ranksep': 4.0, 'fontsize': 12, 'shape': "none", 'width': 1}, 
    display_counts=False, 
    return_raw=True
)
#export_to_tex(mod_freq_tree, "freq_event_tree")
#Image(mod_freq_tree.create_png())

converter = t2t.converter({
  'model': mod_freq_tree,
  'labels': ("Age", "Sex", "Frequency", "Accident"),
  'layer_nodes': (0, 3, 9, 27),
  'filename': "freq_event_tree"
})

converter.export_processed()

In [40]:
mod_freq_staged_tree = mod_freq.staged_tree_figure(
    "figures/tree", 
    params={'ranksep': 4, 'fontsize': 12, 'shape': "circle", 'width': 1, 'style': "filled"}, 
    display_counts=False,
    return_raw=True
)
export_to_tex(mod_freq_staged_tree, "freq_staged_tree")
#Image(mod_freq_staged_tree .create_png())

In [5]:
mod_freq_ceg = mod_freq.ceg_figure_optimal(
    "ceg",  
    display_probs=True, 
    return_raw=True,
    params = {'ranksep': 3, 'fontsize': 15, 'shape': "circle", 'width': 1, 'style': "filled"},
)
export_to_tex(mod_freq_ceg, "freq_ceg")
#Image(mod_freq_ceg.create_png())

# Mileage model - complete observations

In [6]:
df = pd.read_csv("../data/processed/miles_mod.csv", index_col=0)
df.head()

,age_group,sex,miles,acc_inv
1,"(18,21]",M,3.0,0
2,"(21,80]",F,3.0,0
3,"(21,80]",M,3.0,0
4,"(21,80]",F,2.0,0
5,"(21,80]",F,3.0,0


In [7]:
df["miles"].isna().sum()
df.count()

age_group    9491
sex          9491
miles        8734
acc_inv      9491
dtype: int64

In [8]:
order = ["age_group", "sex", "miles", "acc_inv"]
mod_miles = tree.ceg({'dataframe' : df[order].dropna()})
new_nodes = ["w_{" + str(n) + "}" for n in list(range(0, len(mod_miles.nodes)))]
mod_miles.rename_nodes(new_nodes)
mod_miles.AHC_transitions(alpha = 3)
mod_miles_ceg = mod_miles.ceg_figure_optimal(
    "ceg", params = {'ranksep' : 2, 'fontsize': 15, 'shape': "circle", 'width': 0.5, 'style': "filled"}, 
    display_probs=True, 
    return_raw=True
)
export_to_tex(mod_miles_ceg, "miles_complete_ceg")
#Image(mod_miles_ceg.create_png())

# Mileage model - missing values

In [9]:
order = ["age_group", "sex", "is_missing", "miles", "acc_inv"]
df["is_missing"] = np.where(df["miles"].isna(), "yes", "no")
df["miles"] = np.where(df["miles"].isna(), np.nan, df["miles"].astype('Int64'))
mod_miles = tree.ceg({'dataframe' : df[order]})
new_nodes = ["w_{" + str(n) + "}" for n in list(range(0, len(mod_miles.nodes)))]
mod_miles.rename_nodes(new_nodes)
mod_miles.AHC_transitions(alpha = 3)
mod_miles_ceg = mod_miles.ceg_figure_optimal(
    "ceg", params = {'ranksep' : 1, 'fontsize': 15, 'shape': "circle", 'width': 0.5, 'style': "filled"}, 
    display_probs=True, 
    return_raw=True,
)
export_to_tex(mod_miles_ceg, "miles_missing_ceg")
#Image(mod_miles_ceg.create_png())

# Missing responses

In [14]:
df = pd.read_csv("../data/processed/new_missing_values.csv", index_col = 0)
df = df[df["sent"] == 1]
df.head()

,sex,cohort,sent,wave1,wave2,wave3,wave4
24,F,2,1,1,1,1.0,1.0
25,M,15,1,1,1,NaN,NaN
41,F,5,1,0,0,0.0,0.0
51,M,3,1,0,1,1.0,1.0
54,M,4,1,1,0,1.0,1.0


In [15]:
df.groupby("sex").sum()

,cohort,sent,wave1,wave2,wave3,wave4
sex,,,,,,
F,96574,11817,6410,4825,2792.0,1878.0
M,65477,8115,3640,2613,1394.0,880.0


In [18]:
order = ["sex", "wave1", "wave2", "wave3", "wave4"]
mod_responses = tree.ceg({'dataframe' : df[order]})
new_nodes = ["w_{" + str(n) + "}" for n in list(range(0, len(mod_responses.nodes)))]
mod_responses.rename_nodes(new_nodes)
mod_responses.AHC_transitions(alpha = 2)
mod_responses_ceg = mod_responses.ceg_figure_optimal("plot_ceg", return_raw=True)
export_to_tex(mod_responses_ceg, "responses_ceg")
#Image(mod_responses_ceg.create_png())

In [21]:
df["12"] = df["wave1"].astype(str) + df["wave2"].astype(str)
order = ["sex", "12", "wave3", "wave4"]
mod_compact_responses = tree.ceg({'dataframe' : df[order]})
new_nodes = ["w_{" + str(n) + "}" for n in list(range(0, len(mod_compact_responses.nodes)))]
mod_compact_responses.rename_nodes(new_nodes)
mod_compact_responses.AHC_transitions(alpha = 2)
mod_responses_compact_ceg = mod_responses.ceg_figure_optimal("plot_ceg", return_raw=True)
export_to_tex(mod_responses_compact_ceg, "responses_compact_ceg")
#Image(mod_responses_ceg.create_png())